# Graph

In this tutorial we introduce the notion of `MultiModalGraph`, how to construct and manipulate it.

`MultiModalGraph` is made of two component, the `flow_graph` and the `mobility_graph`. They both are an `OrientedGraph`, but one is geometric and the other topologic. 
The `flow_graph` is a geometric representation of the roads. The `mobility_graph` represent each mobility service define on the `flow_graph`.

## Create a simple flow graph

### Instantiation

In [ ]:
from mnms.graph import MultiModalGraph

mmgraph  = MultiModalGraph()
flow_graph = mmgraph.flow_graph

### Adding nodes

In [ ]:
flow_graph.add_node('IN', [-50, 0])
flow_graph.add_node('0', [0, 0])
flow_graph.add_node('1', [100, 0])
flow_graph.add_node('2', [100, 100])
flow_graph.add_node('3', [0, 100])
flow_graph.add_node('OUT', [150, 100]) 

flow_graph.nodes

### Adding links

In [ ]:
flow_graph.add_link('IN_0', 'IN', '0')
flow_graph.add_link('0_1', '0', '1')
flow_graph.add_link('1_2', '1', '2')
flow_graph.add_link('0_3', '0', '3')
flow_graph.add_link('3_2', '3', '2')
flow_graph.add_link('2_OUT', '2', 'OUT')

flow_graph.links

### Rendering

In [ ]:
from mnms.tools.render import draw_flow_graph
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 9))
draw_flow_graph(ax, flow_graph, nodesize=5, linkwidth=2)

## Add mobility services

We add the nodes and links for the car and bus mobility service, each `TopoNode` in a mobility service can refer to a `GeoNode` of the `flow_graph`.
Each link define a dictionnary of cost, in this example we use the travel time of the links.

### Car layer with a personal car mobility service

In [ ]:
from mnms.mobility_service.car import CarMobilityGraphLayer, PersonalCarMobilityService

SPEED_CAR = 10

car = CarMobilityGraphLayer('CarLayer', 
                            SPEED_CAR,
                            services=[PersonalCarMobilityService()])

car.add_node('CIN', 'IN')
car.add_node('C0', '0')
car.add_node('C1', '1')
car.add_node('C2', '2')
car.add_node('COUT', 'OUT')

car.add_link('CIN_C0', 'CIN', 'C0', ['IN_0'], {'travel_time':0.5/SPEED_CAR})
car.add_link('C0_C1', 'C0', 'C1', ['0_1'], {'travel_time':1/SPEED_CAR})
car.add_link('C1_C2', 'C1', 'C2', ['1_2'], {'travel_time':1/SPEED_CAR})
car.add_link('C2_COUT', 'C2', 'COUT', ['2_OUT'], {'travel_time':0.5/SPEED_CAR})

car.mobility_services

### Bus layer with apublic transportmobility service

In [ ]:
from mnms.mobility_service.public_transport import BusMobilityGraphLayer, PublicTransportMobilityService
from mnms.tools.time import TimeTable, Dt

SPEED_BUS = 5

bus = BusMobilityGraphLayer('BusLayer',
                            SPEED_BUS,
                            services=[PublicTransportMobilityService('Bus')])

bus_line = bus.add_line('L1', TimeTable.create_table_freq('07:00:00', '18:00:00', Dt(minutes=10)))

bus_line.add_stop('BIN', 'IN')
bus_line.add_stop('B0', '0')
bus_line.add_stop('B3', '3')
bus_line.add_stop('B2', '2')
bus_line.add_stop('BOUT', 'OUT')

bus_line.connect_stops('BIN_B0', 'BIN', 'B0', 0.5, ['IN_0'], {'travel_time':0.5/SPEED_BUS})
bus_line.connect_stops('B0_B3', 'B0', 'B3', 1, ['0_3'], {'travel_time':1/SPEED_BUS})
bus_line.connect_stops('B3_B2', 'B3', 'B2', 1, ['3_2'], {'travel_time':1/SPEED_BUS})
bus_line.connect_stops('B2_COUT', 'B2', 'BOUT', 0.5, ['2_OUT'], {'travel_time':0.5/SPEED_BUS})

bus.mobility_services

### Add the mobility services to the MultiModalGraph

Then we add the two mobility services to the `MultiModalGraph`. And check that there is no duplicate in the nodes and links.

In [ ]:
mmgraph.add_layer(car)
mmgraph.add_layer(bus)
mmgraph.mobility_graph.check()

mmgraph.layers

### Rendering the mobility services

In [ ]:
from mnms.tools.render import draw_multimodal_graph

fig, ax = plt.subplots(figsize=(16, 9))
draw_multimodal_graph(ax, mmgraph, nodesize=5, linkwidth=2, dy=50)

## Compute shortest path

To compute a shortest path, we define a `User` with an origin/destination and a departure time. The shortest path algorithm use the dictionary of cost define on links. We use the `time` cost previously define.

In [ ]:
from mnms.graph.shortest_path import compute_shortest_path
from mnms.demand.user import User
from mnms.tools.time import Time

user = User('U0', 'IN', 'OUT', Time('07:00:00'))
path = compute_shortest_path(mmgraph, user)
print(f"Path cost: {path.path_cost}, path:{path.nodes}") 

## Save graph

You can save the `MultiModalGraph` in a json format using the `io` module.

In [ ]:
from mnms.graph.io import save_graph

save_graph(mmgraph, 'results/mmgraph.json')

And reload it using the `load_graph` function:

In [ ]:
from mnms.graph.io import load_graph

new_mmgraph = load_graph('results/mmgraph.json')

fig, ax = plt.subplots(figsize=(16, 9))
draw_multimodal_graph(ax, new_mmgraph, nodesize=5, linkwidth=2, dy=50)